In [2]:
import mediapipe as mp
import cv2
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from IPython.display import clear_output 

objc[34828]: Class CaptureDelegate is implemented in both /Users/vaisakh/miniforge3/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x118e7c860) and /Users/vaisakh/miniforge3/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x2907965a0). One of the two will be used. Which one is undefined.
objc[34828]: Class CVWindow is implemented in both /Users/vaisakh/miniforge3/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x107794a68) and /Users/vaisakh/miniforge3/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x2907965f0). One of the two will be used. Which one is undefined.
objc[34828]: Class CVView is implemented in both /Users/vaisakh/miniforge3/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x107794a90) and /Users/vaisakh/miniforge3/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x290796618). One of the two will be used. Which one is undefined.
objc[34828]: Class CVSlider is implemented in both /Users/v

In [3]:
model = load_model("/Users/vaisakh/research_final_codebase/mp_last_large_map")

2023-03-31 11:39:22.274366: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-31 11:39:22.274760: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



In [4]:
def preprocess(landmark):
    processed_landmarks = []
    relative_landmarks= []
    for i in landmark[0].landmark:
        processed_landmarks.append([i.x, i.y])

    for i in processed_landmarks:
        relative_landmarks.append([
            i[0]-processed_landmarks[0][0],
            i[1]-processed_landmarks[0][1],
        ])
    
    flatten = []
    for i in relative_landmarks:
        flatten.append(i[0])
        flatten.append(i[1])
    
    maxVal = max(list(map(abs, flatten)))
    def norm(n): return n/maxVal
    normalised = list(map(norm, flatten))

    return normalised

In [5]:
encoding_maps = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 
       'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'Space', 'T', 
       'U', 'V', 'W', 'X', 'Y', 'Z']

In [16]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
cap = cv2.VideoCapture(1)
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands: 
    while cap.isOpened():
    # while True:
        ret, frame = cap.read()
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = cv2.flip(image, 1)
        image.flags.writeable = False
        results = hands.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                    mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                            mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                            mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
                                        )

            res = preprocess(results.multi_hand_landmarks)
            prob = model.predict([res])
            pred = np.argmax(prob)
            maxi = prob[0][pred]
            tresh = .98
            if maxi < tresh: continue
            sign = encoding_maps[pred]
            print("res:", sign, "max - prob:", maxi)
            image = cv2.putText(image, sign, org=(50, 50), 
                                fontFace=cv2.FONT_HERSHEY_SIMPLEX, 
                                fontScale=1, color=(0,0,0), 
                                thickness=2)

        cv2.imshow('Hand Tracking', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)

1/1 [==============================] - 0s 20ms/step
res: B max - prob: 0.9854746
1/1 [==============================] - 0s 20ms/step
res: B max - prob: 0.99003756
1/1 [==============================] - 0s 19ms/step
res: B max - prob: 0.99048156
1/1 [==============================] - 0s 19ms/step
res: B max - prob: 0.99165386
1/1 [==============================] - 0s 19ms/step
res: B max - prob: 0.9912636
1/1 [==============================] - 0s 19ms/step
res: B max - prob: 0.99137926
1/1 [==============================] - 0s 19ms/step
res: B max - prob: 0.9920489
1/1 [==============================] - 0s 19ms/step
res: B max - prob: 0.9919993
1/1 [==============================] - 0s 19ms/step
res: B max - prob: 0.9916346
1/1 [==============================] - 0s 19ms/step
res: B max - prob: 0.99176395
1/1 [==============================] - 0s 19ms/step
res: B max - prob: 0.991738
1/1 [==============================] - 0s 18ms/step
res: B max - prob: 0.99189126
1/1 [==================

-1